##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [5]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from functools import partial
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
importlib.reload(Dense)
importlib.reload(RSTF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats

from ipywidgets import FloatProgress, HTML
from IPython.display import display, clear_output
import os  
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope):
    if Scope == 'Full':
        Model = ['PPFD_Avg','AirTC_Avg','VPD','wind_speed',
                'Temp','VWC','Sedge','Shrub','Grass','Upland',
                'HR','DOY']
    if Scope == 'Test':
        Model = ['PPFD_Avg','VPD','wind_speed','VWC']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
#     print('Models: ',Models)
    return(Models)

def Stats(mse,se,r2,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse],
                            'Size':j,
                            'Number':i,
                            'HiddenNodes':params['N'],
                            'SE':[se],
                            'r2':[r2],
                            'Performance':0,
                            'K':[params['K']]})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))

FirstRun = True
# FirstRun = False

MP=True
Scope = 'Full'
# Scope = 'Test'
processes=3
    
cwd = os.getcwd()
Path='Data'
alpha = .05


pd.set_option('max_colwidth',200)
def Display (tar,prog1,prog2,MdLs,MdL):
    clear_output()
    display(tar)
    display(prog1)
    display(prog2)
    display(MdLs)
    display(MdL)
    
tar = HTML(
            value=" ",
            placeholder='Target: ',
            description='Target: ',
        )
kwt = HTML(
        value=str(0),
        placeholder='Quit Score: ',
        description='Quit Score: ',
        )

# Model

In [4]:
depth = 11
Time = time.time()
FirstRun = True
# FirstRun = False
kwit = 0
Pthresh=1
if __name__ == '__main__':
    for target in ['fch4','fco2']:
        XVarriables=ModSelect(Scope)
        print(XVarriables)
        if FirstRun == True:
            start=1
            stop = len(XVarriables)
            if depth < stop:
                stop = depth
            stop += 1#2
            try:shutil.rmtree(cwd+'/'+Path+'/'+target+'/')
            except:pass
            os.mkdir(cwd+'/'+Path+'/'+target+'/')
        else:
            params = Dense.Params(Scope,target,MP)
            params['Dpath'] = cwd+'/'+Path+'/'
            Records = pd.read_csv(cwd+'/'+Path+'/'+target+'_Runs.csv',delimiter = ',',header = 0,index_col=[0])
            start = Records['Size'].max()+1
            Level = Records.loc[Records['Size']==start-1]
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
            
            stop = len(XVarriables)
            if start >1:
                Factors = Level.loc[Level['Performance']>=Pthresh,'Model'].values
                 
        tar.value=target
        prog1 = FloatProgress(min=start, max=stop,description='Running:')
        prog2 = FloatProgress(min=0, max=100,description='Bootstrapping:')
        MdLs = HTML(
            value=" ",placeholder='Models: ',description='Models: ',
        )
        MdL = HTML(
            value=" ",placeholder='Testing: ',description='Testing: ',
        )
        Display (tar,prog1,prog2,MdLs,MdL)

        for j in range(start,stop):         
            if j == 1: Inputs = (Combos(XVarriables,j))
            else: Inputs = (Combos(XVarriables,j,Factors))
            MdLs.value=str(len(Inputs))
            i = 0
            procede = True
            for Input in Inputs:
                if FirstRun == False:
                    if Records.loc[Records['Size']==j].isin({'Number':[i]}).any().any()==True:
                        procede = False
                    else:
                        procede = True
                if procede == True:
                    params = Dense.Params(Scope,target,MP)
                    params['Dpath'] = cwd+'/'+Path+'/'
                    params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
                    try:os.mkdir(params['Spath'])
                    except:pass
                    params['Sname'] = 'Y_'
                    params['Inputs'] = Input
                    MdL.value='#'+str(i)+' '+str(params['Inputs'])
                    Display (tar,prog1,prog2,MdLs,MdL)
                    print('Runtime: ',time.time()-Time)
                    params['Model'] = '+'.join(params['Inputs'])
                    RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
                    if target == 'ER':
                        RST.Master = RST.Master.loc[RST.Master['fco2']>0]
                    RST.Scale(params['target'],params['Inputs']) 
                    y = RST.y*1.0
                    X = RST.X*1.0
                    Ni = len(Input)
                    Ns = y.shape[0]
                    No = 1
                    a = 2
                    params['N']=int(Ns/(a*(Ni+No)))
                    params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
                    Y_hat=[]
                    y_true=[]
                    X_true=[]
                    index=[]
                    ones=[]
                    prog2.value=0
                    pool = Pool(processes=processes,maxtasksperchild=75)
                    for k,results in enumerate(pool.imap(partial(Dense.Bootstrap,params=params,X=X,y=y),range(params['K']))):
                        Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                        y_true.append(RST.YScaled.inverse_transform(results[1]))
                        X_true.append(RST.XScaled.inverse_transform(results[2]))
                        ones.append(results[3])
                        prog2.value=(k+1)/params['K']*100
                    pool.close()
                    Y_hat = np.squeeze(np.asanyarray(Y_hat))
                    y_true = np.squeeze(np.asanyarray(y_true))
                    X_true = np.asanyarray(X_true)
                    ones = np.asanyarray(ones)
                    params['Memory'] = .95
                    pool = Pool(processes=1,maxtasksperchild=75)
                    for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                     Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
                         mse,se,r2 = results
                    pool.close()
                    if i == 0 and FirstRun == True:Level = Stats(mse,se,r2,j,i,params)
                    else:
                        Level = Level.loc[Level['Size']==j]
                        Level = Level.append(Stats(mse,se,r2,j,i,params))
                    print(Level)
                i += 1
                prog1.value=j+i/len(Inputs)
                
            Level['Thresh'] = Level['MSE']+Level['SE']
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
            if j == 1:
                Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*1,'Performance']+=1
                Records = Level
            else:
                if Records['MSE'].min()>Min['MSE'].values[0]:
                    Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*1,'Performance']+=1
                Top = Level.loc[Level['Performance']==Level['Performance'].max()]
                Smalls = Top.nsmallest(3,'MSE')
                Level.loc[Level.MSE.isin(Smalls['MSE'].values)==False,'Performance']-=1
                Records = Records.append(Level)
            Factors = Level.loc[Level['Performance']>=Pthresh,'Model'].values
            Records = Records.reset_index(drop=True)
            Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
            if Scope == 'Full':
                Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
            else:
                Records.to_csv(params['Dpath']+'/'+target+'_Runs2.csv')
            if Level['Performance'].max()<Pthresh and j>1:
                break